In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import LogisticRegression


In [0]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('UScasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [0]:
data = coviddeath.select("Year","Date","Day", "Temp", "Admin2","Lat","Long","Province",col("Case").alias("label"))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)

data.show(5)


+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
2020| 22| 1| 61|Los Angeles|34.30828379|-118.2282411|California| 0| 0.0| 0.0|
2020| 23| 1| 59|Los Angeles|34.30828379|-118.2282411|California| 0| 0.0| 0.0|
2020| 24| 1| 63|Los Angeles|34.30828379|-118.2282411|California| 0| 0.0| 0.0|
2020| 25| 1| 64|Los Angeles|34.30828379|-118.2282411|California| 0| 0.0| 0.0|
2020| 26| 1| 58|Los Angeles|34.30828379|-118.2282411|California| 1| 0.0| 0.0|
+----+----+---+----+-----------+-----------+------------+----------+-----+------------+--------------+
only showing top 5 rows

In [0]:
splits = data.randomSplit([0.7, 0.3])
# for gradient boosted tree regression
dt_train = splits[0]
dt_test = splits[1].withColumnRenamed("label", "trueLabel")

print ("Training Rows:", dt_train.count(), " Testing Rows:", dt_test.count())

dt_train.show(5)


Training Rows: 300 Testing Rows: 136
+----+----+---+----+------------+-----------+------------+------------+-----+------------+--------------+
Year|Date|Day|Temp| Admin2| Lat| Long| Province|label|Admin2_index|Province_index|
+----+----+---+----+------------+-----------+------------+------------+-----+------------+--------------+
2020| 1| 2| 43| New York| 40.7672726|-73.97152637| New York| 0| 1.0| 1.0|
2020| 1| 2| 44|Philadelphia|40.00338507| -75.1379271|Pennsylvania| 0| 2.0| 2.0|
2020| 1| 2| 56| Spokane|47.62113146|-117.4046494| Washington| 0| 3.0| 3.0|
2020| 1| 3| 58| Los Angeles|34.30828379|-118.2282411| California| 1| 0.0| 0.0|
2020| 1| 3| 60|Philadelphia|40.00338507| -75.1379271|Pennsylvania| 0| 2.0| 2.0|
+----+----+---+----+------------+-----------+------------+------------+-----+------------+--------------+
only showing top 5 rows

In [0]:

#assembler = VectorAssembler(inputCols =["Day","Temp","Lat","Long","Admin_index","Province_index"],outputCol="features")
assembler = VectorAssembler(inputCols =["Date","Year","Day","Temp","Lat","Admin2_index","Province_index"],outputCol="features")
#assembler = VectorAssembler(inputCols =["Date","Year","Day","Temp"],outputCol="features")

dt = DecisionTreeRegressor(featuresCol='features', labelCol='label', maxBins=77582)
stages = [assembler, dt]
pipeline = Pipeline(stages=stages)
#dt_pipeline = pipeline(stages=[assembler, dt])
pipelineModel = pipeline.fit(dt_train)
dtModel = pipelineModel.stages[-1]
print(dtModel.toDebugString)

#paramGrid = ParamGridBuilder().build()
#cv = CrossValidator(estimator=dt_pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=5)
#dt_model = cv.fit(dt_train)
#dt_prediction = dt_model.transform(dt_test)
#dt_predicted = dt_prediction.select("features", "prediction", "trueLabel")
#dt_predicted.show(10)



DecisionTreeRegressionModel: uid=DecisionTreeRegressor_6f50c4989dce, depth=5, numNodes=59, numFeatures=7
 If (feature 5 in {0.0,2.0,3.0})
 If (feature 2 <= 3.5)
 If (feature 2 <= 2.5)
 If (feature 4 <= 37.15583443)
 If (feature 2 <= 1.5)
 Predict: 0.5555555555555556
 Else (feature 2 > 1.5)
 Predict: 1.0
 Else (feature 4 > 37.15583443)
 Predict: 0.0
 Else (feature 2 > 2.5)
 If (feature 0 <= 28.5)
 If (feature 0 <= 23.5)
 Predict: 49.851063829787236
 Else (feature 0 > 23.5)
 Predict: 425.53846153846155
 Else (feature 0 > 28.5)
 If (feature 4 <= 37.15583443)
 Predict: 1829.0
 Else (feature 4 > 37.15583443)
 Predict: 1092.3333333333333
 Else (feature 2 > 3.5)
 If (feature 4 <= 40.385328834999996)
 If (feature 2 <= 4.5)
 If (feature 0 <= 20.5)
 Predict: 6726.884615384615
 Else (feature 0 > 20.5)
 Predict: 15326.466666666667
 Else (feature 2 > 4.5)
 If (feature 4 <= 37.15583443)
 Predict: 28607.333333333332
 Else (feature 4 > 37.15583443)
 Predict: 16154.57142857143
 Else (feature 4 > 40.385328834999996)
 If (feature 2 <= 4.5)
 If (feature 0 <= 16.5)
 Predict: 251.4
 Else (feature 0 > 16.5)
 Predict: 328.6666666666667
 Else (feature 2 > 4.5)
 If (feature 0 <= 5.5)
 Predict: 369.5
 Else (feature 0 > 5.5)
 Predict: 380.3333333333333
 Else (feature 5 not in {0.0,2.0,3.0})
 If (feature 2 <= 3.5)
 If (feature 0 <= 22.5)
 If (feature 0 <= 19.5)
 If (feature 0 <= 12.5)
 Predict: 13.9375
 Else (feature 0 > 12.5)
 Predict: 192.5
 Else (feature 0 > 19.5)
 If (feature 2 <= 2.5)
 Predict: 0.0
 Else (feature 2 > 2.5)
 Predict: 6340.5
 Else (feature 0 > 22.5)
 If (feature 2 <= 2.5)
 Predict: 0.0
 Else (feature 2 > 2.5)
 If (feature 0 <= 25.5)
 Predict: 15021.666666666666
 Else (feature 0 > 25.5)
 Predict: 25740.666666666668
 Else (feature 2 > 3.5)
 If (feature 2 <= 4.5)
 If (feature 0 <= 14.5)
 If (feature 0 <= 7.5)
 Predict: 62169.166666666664
 Else (feature 0 > 7.5)
 Predict: 97137.0
 Else (feature 0 > 14.5)
 If (feature 0 <= 23.5)
 Predict: 136271.0
 Else (feature 0 > 23.5)
 Predict: 160207.8
 Else (feature 2 > 4.5)
 If (feature 0 <= 4.5)
 If (feature 0 <= 2.5)
 Predict: 171022.0
 Else (feature 0 > 2.5)
 Predict: 175651.0
 Else (feature 0 > 4.5)
 If (feature 0 <= 6.5)
 Predict: 177612.5
 Else (feature 0 > 6.5)
 Predict: 181762.66666666666

In [0]:
import pandas as pd
featureImp = pd.DataFrame(list(zip(assembler.getInputCols(), dtModel.featureImportances)),columns=["feature", "importance"])
featureImp.sort_values(by="importance", ascending=False)

Out[17]:

,feature,importance
2,Day,0.609987
5,Admin2_index,0.320423
0,Date,0.064503
4,Lat,0.005087
1,Year,0.000000
3,Temp,0.000000
6,Province_index,0.000000


In [0]:
dt_evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
dt_rmse = dt_evaluator.evaluate(dt_prediction)


print ("Root Mean Square Error (RMSE):", dt_rmse)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(dt_prediction)
print(accuracy)